<a href="https://colab.research.google.com/github/JonJonesBR/MESTRE_RPG_GEMINI/blob/main/RPG_AUDIO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install -q google-generativeai edge-tts playsound nest_asyncio

import google.generativeai as genai
import time
import os
import io # Para manipulação de bytes de áudio em memória
import sys # Para verificar se está rodando no Colab
import asyncio

# Configuração para permitir loops asyncio aninhados (crucial para Jupyter/Colab)
try:
    import nest_asyncio
    nest_asyncio.apply()
    print("nest_asyncio aplicado.")
except ImportError:
    print("AVISO: nest_asyncio não encontrado. Se ocorrerem erros de 'event loop is already running', instale-o: pip install nest_asyncio")
except RuntimeError:
    print("AVISO: nest_asyncio já pode estar aplicado ou houve um erro ao aplicá-lo.")


try:
    import edge_tts
    _edge_tts_available = True
except ImportError:
    _edge_tts_available = False
    print("AVISO: Biblioteca edge-tts não encontrada. A funcionalidade de voz não estará disponível. Instale com: pip install edge-tts")

try:
    import IPython.display as ipd
    _ipython_available = True
except ImportError:
    _ipython_available = False

_IS_COLAB = 'google.colab' in sys.modules and _ipython_available

API_KEY_FILENAME = ".gemini_api_key.txt"

DURATION_OPTIONS_CONFIG = {
    "1": {"name": "Curta (objetivo direto)", "id": "curta", "turns": 10},
    "2": {"name": "Média (mais exploração)", "id": "media", "turns": 20},
    "3": {"name": "Longa (desenvolvimento e clímax)", "id": "longa", "turns": 40}
}

def get_and_configure_api_key():
    api_key = None
    if os.path.exists(API_KEY_FILENAME):
        try:
            with open(API_KEY_FILENAME, "r") as f: api_key = f.read().strip()
            if api_key and api_key.startswith("AIza") and len(api_key) > 20: print("Chave API carregada de arquivo local.")
            else:
                api_key = None
                if os.path.exists(API_KEY_FILENAME):
                    try:
                        os.remove(API_KEY_FILENAME)
                        print(f"Arquivo '{API_KEY_FILENAME}' continha uma chave inválida e foi removido.")
                    except Exception as e_rem: print(f"Não foi possível remover o arquivo de chave inválida: {e_rem}")
        except Exception as e:
            print(f"Erro ao ler a chave API do arquivo '{API_KEY_FILENAME}': {e}")
            api_key = None
    if not api_key:
        print("\n--- Configuração da Chave API do Gemini ---")
        print("Para jogar, você precisará de uma chave API do Google Gemini.")
        print("Você pode obter uma gratuitamente em: https://aistudio.google.com/app/apikey")
        while True:
            entered_key = input("Por favor, insira sua chave API do Gemini: ").strip()
            if entered_key.startswith("AIza") and len(entered_key) > 20:
                api_key = entered_key
                try:
                    with open(API_KEY_FILENAME, "w") as f: f.write(api_key)
                    print(f"Chave API válida recebida e salva localmente em '{API_KEY_FILENAME}'.")
                    print("Você não precisará inseri-la novamente neste ambiente se o arquivo persistir.")
                except Exception as e:
                    print(f"Não foi possível salvar a chave API: {e}. Usando apenas para esta sessão.")
                break
            else:
                print("Formato de chave API inválido.")
                if input("Tentar novamente? (S/N): ").strip().upper() != 'S':
                    print("Sem chave API válida, o jogo não pode continuar.")
                    return False
    if not api_key: print("Chave API não configurada."); return False
    try:
        genai.configure(api_key=api_key)
        print("API do Gemini configurada com sucesso.")
        return True
    except Exception as e:
        print(f"Ocorreu um erro ao configurar a API Gemini: {e}")
        if os.path.exists(API_KEY_FILENAME):
            try:
                os.remove(API_KEY_FILENAME)
                print(f"Arquivo '{API_KEY_FILENAME}' com chave inválida removido.")
            except Exception as e_rem: print(f"Não foi possível remover o arquivo de chave inválida: {e_rem}")
        return False

if not get_and_configure_api_key():
    raise SystemExit("Falha na configuração da API Key. Encerrando o programa.")

generation_config = {"temperature": 0.8, "top_p": 0.95, "top_k": 40}
safety_settings = [
    {"category": c, "threshold": "BLOCK_MEDIUM_AND_ABOVE"}
    for c in ["HARM_CATEGORY_HARASSMENT", "HARM_CATEGORY_HATE_SPEECH", "HARM_CATEGORY_SEXUALLY_EXPLICIT", "HARM_CATEGORY_DANGEROUS_CONTENT"]
]
model = genai.GenerativeModel(model_name="gemini-1.5-flash-latest", generation_config=generation_config, safety_settings=safety_settings)
print(f"Modelo Gemini ({model.model_name}) configurado!")

game_state = {
    "player_name": None, "player_class": None, "player_inventory": [], "current_location": None,
    "current_situation": None, "npcs_encountered": {}, "plot_twists_triggered": [],
    "previous_player_actions": [], "world_lore": {"setting_description": None, "key_events_history": []},
    "current_objective": None, "chosen_duration_id": None, "chosen_duration_name": None,
    "max_turns": 0, "current_turn": 0
}
conversation_history = []

async def _speak_text_async(text, voice_name="pt-BR-ThalitaNeural", rate="+0%", pitch="+0Hz"):
    if not _edge_tts_available or not text or not text.strip(): return
    communicate = None
    try:
        communicate = edge_tts.Communicate(text, voice_name, rate=rate, pitch=pitch)
        if _IS_COLAB:
            audio_bytes_io = io.BytesIO()
            async for chunk in communicate.stream():
                if chunk["type"] == "audio": audio_bytes_io.write(chunk["data"])
            audio_bytes_io.seek(0)
            if audio_bytes_io.getbuffer().nbytes > 0:
                ipd.display(ipd.Audio(data=audio_bytes_io.getvalue(), autoplay=True))
                estimated_duration_sec = len(text) * 0.075
                sleep_duration = min(max(1.0, estimated_duration_sec), 12.0)
                await asyncio.sleep(sleep_duration)
            else: print("[TTS Edge Erro: Nenhum dado de áudio gerado para Colab.]")
        else:
            temp_mp3_filename = ".temp_edge_speech.mp3"
            await communicate.save(temp_mp3_filename)
            try:
                from playsound import playsound
                playsound(temp_mp3_filename, True)
            except ImportError: print(f"\n[AVISO TTS: 'playsound' não instalado. Áudio salvo em '{temp_mp3_filename}']\n")
            except Exception as e_play: print(f"\n[ERRO TTS: Falha ao tocar áudio local - {e_play}. Áudio em '{temp_mp3_filename}']\n")
            finally:
                if os.path.exists(temp_mp3_filename):
                    try: os.remove(temp_mp3_filename)
                    except Exception: pass
    except edge_tts.exceptions.NoAudioGenerated as e_no_audio:
        print(f"[TTS Edge Erro: Nenhum áudio gerado - {e_no_audio}]")
        if communicate and hasattr(communicate, 'error_reason') and communicate.error_reason: print(f"Razão do erro edge_tts: {communicate.error_reason}")
    except Exception as e_tts: print(f"[TTS Edge Erro Geral: {e_tts}]")

def speak_text(text, voice_name="pt-BR-ThalitaNeural"):
    if not _edge_tts_available or not text or not text.strip(): return
    try: asyncio.run(_speak_text_async(text, voice_name))
    except RuntimeError as e:
        if "cannot be called when another event loop is running" in str(e): print(f"[AVISO TTS: Loop asyncio já rodando. Erro: {e}]")
        else: print(f"[ERRO TTS (asyncio runtime): {e}]")
    except Exception as e_gen: print(f"[ERRO TTS (chamada síncrona): {e_gen}]")

def mestre_responde(prompt_para_o_mestre):
    try:
        time.sleep(1)
        response = model.generate_content(prompt_para_o_mestre)
        return response.text
    except Exception as e:
        if "429" in str(e) or "quota" in str(e).lower(): return f"Desculpe, aventureiro, energias místicas sobrecarregadas (cota API). (Erro: {e})"
        return f"Desculpe, aventureiro, energias místicas instáveis. (Erro: {e})"

def parse_initial_setup_from_gemini(response_text):
    data = {
        "current_objective": "Investigar os mistérios nos arredores e encontrar um caminho seguro.",
        "current_location": "Um lugar enigmático envolto por uma névoa densa e desconhecida.",
        "player_inventory": ["Um cantil com um gole d'água", "Um mapa rudimentar da região (ilegível)"]
    }
    try:
        if not response_text or not response_text.strip():
            print("Resposta da IA para setup inicial vazia. Usando valores padrão.")
            return data
        lines = response_text.strip().split('\n')
        found_objective, found_location, found_inventory = False, False, False
        for line in lines:
            line_upper = line.upper()
            if line_upper.startswith("OBJETIVO:"): data["current_objective"] = line.split(":", 1)[1].strip(); found_objective = True
            elif line_upper.startswith("LOCALIZAÇÃO:") or line_upper.startswith("LOCALIZACAO:"): data["current_location"] = line.split(":", 1)[1].strip(); found_location = True
            elif line_upper.startswith("INVENTÁRIO:") or line_upper.startswith("INVENTARIO:"):
                items_str = line.split(":", 1)[1].strip().replace('[','').replace(']','')
                parsed_items = [item.strip().strip("'\"") for item in items_str.split(',') if item.strip()]
                if parsed_items: data["player_inventory"] = parsed_items; found_inventory = True
        if not found_objective: data["current_objective"] = "Explorar a área e descobrir seu propósito."
        if not found_location: data["current_location"] = "Um ponto de partida misterioso."
        if not found_inventory or not data["player_inventory"]: data["player_inventory"] = ["Suprimentos básicos", "Ferramenta útil"]
    except Exception as e:
        print(f"Erro ao processar dados iniciais da IA para setup: {e}. Usando valores padrão.")
        data = { # Garante retorno dos defaults em caso de exceção no parsing
            "current_objective": "Investigar os mistérios nos arredores e encontrar um caminho seguro.",
            "current_location": "Um lugar enigmático envolto por uma névoa densa e desconhecida.",
            "player_inventory": ["Um cantil com um gole d'água", "Um mapa rudimentar da região (ilegível)"]
        }
    return data

def iniciar_aventura():
    global game_state, conversation_history
    welcome_message = "Bem-vindo, aventureiro! Antes de começarmos nossa jornada, conte-me sobre você."
    print(welcome_message); speak_text(welcome_message)
    player_name_input = input("Qual é o seu nome, ó valente? ")
    game_state["player_name"] = player_name_input.strip() if player_name_input.strip() else "Aventureiro Anônimo"
    if game_state["player_name"] == "Aventureiro Anônimo":
        msg_anon = "Vejo que prefere o mistério... 'Aventureiro Anônimo' será seu título."; print(msg_anon); speak_text(msg_anon)

    player_class_options = ["Guerreiro Astuto", "Maga Perspicaz", "Ladina Silenciosa", "Explorador Corajoso", "Bardo Eloquente"]
    msg_class_choice = "\nEscolha sua vocação entre os heróis de outrora, ou invente a sua:" # Mensagem atualizada
    print(msg_class_choice); speak_text(msg_class_choice)
    for i, cls in enumerate(player_class_options): print(f"{i+1}. {cls}")
    print(f"Ou simplesmente digite o nome da classe que deseja inventar.") # Instrução para classe personalizada

    class_choice_valid = False
    while not class_choice_valid:
        choice_input_str = input(f"Digite o número de uma classe da lista ou o nome da sua classe personalizada: ").strip()
        if not choice_input_str:
            print("Por favor, digite uma opção válida.")
            continue
        try:
            choice_idx_plus_1 = int(choice_input_str) # Tenta converter para número
            if 1 <= choice_idx_plus_1 <= len(player_class_options):
                game_state["player_class"] = player_class_options[choice_idx_plus_1 - 1]
                class_choice_valid = True
            else:
                # Número digitado, mas fora do range das opções válidas
                print(f"Número inválido. Escolha de 1 a {len(player_class_options)} ou digite um nome de classe.")
        except ValueError:
            # Não foi um número, então considera como classe personalizada
            game_state["player_class"] = choice_input_str
            class_choice_valid = True
            print(f"Você definiu sua classe como: {game_state['player_class']}") # Feedback para classe personalizada

    msg_class_chosen = f"Sua classe é: {game_state['player_class']}."; print(msg_class_chosen); speak_text(msg_class_chosen)

    msg_duration_choice = "\nEscolha a duração desejada para esta aventura:"; print(msg_duration_choice); speak_text(msg_duration_choice)
    for k, v in DURATION_OPTIONS_CONFIG.items(): print(f"{k}. {v['name']} (~{v['turns']} turnos)")
    duration_choice_valid = False
    while not duration_choice_valid:
        choice_key = input(f"Digite o número (1-{len(DURATION_OPTIONS_CONFIG)}): ")
        if choice_key in DURATION_OPTIONS_CONFIG:
            opt = DURATION_OPTIONS_CONFIG[choice_key]
            game_state.update({"chosen_duration_id": opt['id'], "chosen_duration_name": opt['name'], "max_turns": opt['turns']})
            duration_choice_valid = True
        else: print("Escolha inválida.")
    msg_dur_set = f"Duração: {game_state['chosen_duration_name']}."; print(msg_dur_set); speak_text(msg_dur_set)

    msg_gen_setup = "\nMestre (Gemini): Definindo os contornos iniciais da sua saga..."; print(msg_gen_setup); speak_text(msg_gen_setup)
    prompt_para_setup_inicial = f"""
    Para uma nova aventura de RPG com o jogador '{game_state['player_name']}' (classe: {game_state['player_class']}), que deseja uma aventura '{game_state['chosen_duration_name']}', gere os seguintes elementos de partida:
    1. OBJETIVO: Um objetivo inicial claro, curto e intrigante para o jogador.
    2. LOCALIZAÇÃO: Uma breve descrição (1-2 frases) do local onde o jogador se encontra inicialmente.
    3. INVENTÁRIO: Uma lista de 2 a 3 itens iniciais básicos que o jogador possui, adequados à sua classe e ao cenário/objetivo. Formate como: item1, item2, item3

    Responda APENAS com os elementos solicitados, usando os prefixos OBJETIVO:, LOCALIZAÇÃO:, INVENTÁRIO: em linhas separadas e distintas.
    Exemplo de formato de resposta:
    OBJETIVO: Encontrar a fonte dos sussurros na Floresta Murmurante.
    LOCALIZAÇÃO: Uma cabana abandonada na orla da Floresta Murmurante, com a porta rangendo ao vento.
    INVENTÁRIO: Espada enferrujada, Pederneira, Um pedaço de pão seco
    """
    resposta_setup_inicial = mestre_responde(prompt_para_setup_inicial)
    if "Erro:" in resposta_setup_inicial:
        err_msg = f"\nFalha ao gerar dados iniciais da IA para o setup: {resposta_setup_inicial}"; print(err_msg); speak_text(f"Falha ao gerar dados iniciais da Inteligência Artificial. {resposta_setup_inicial}")
        fallback_msg = "Iniciando com valores padrão para objetivo, localização e inventário."; print(fallback_msg); speak_text(fallback_msg)
        initial_data = parse_initial_setup_from_gemini("")
    else: initial_data = parse_initial_setup_from_gemini(resposta_setup_inicial)

    game_state["current_objective"] = initial_data.get("current_objective", "Investigar e descobrir seu caminho.")
    game_state["current_location"] = initial_data.get("current_location", "Um local desconhecido.")
    game_state["player_inventory"] = initial_data.get("player_inventory", ["Algo útil", "Provisões"])
    game_state["current_turn"] = 0

    print(f"\nSaudações, {game_state['player_name']}, o(a) {game_state['player_class']}!") # Mantido "(o/a)" para generalidade
    speak_text(f"Saudações, {game_state['player_name']}, o {game_state['player_class']}!")
    # print(f"Você escolheu uma aventura: {game_state['chosen_duration_name']}.") # Já falado

    obj_msg=f"\nSeu objetivo: {game_state['current_objective']}"; print(obj_msg); speak_text(obj_msg)
    loc_msg=f"Você se encontra em: {game_state['current_location']}"; print(loc_msg); speak_text(loc_msg)
    inv_msg=f"Em sua bolsa, você carrega: {', '.join(game_state['player_inventory'] if game_state['player_inventory'] else ['nada de especial'])}"; print(inv_msg); speak_text(inv_msg)

    start_msg="\nO ar crepita com magia ancestral... Prepare-se, pois sua aventura está prestes a começar!\n"; print(start_msg); speak_text(start_msg)

    prompt_inicial_contexto = f"""
    Você é um Mestre de RPG (Game Master - GM) experiente, narrando uma aventura de fantasia épica e imersiva.
    O jogador é '{game_state['player_name']}', um(a) {game_state['player_class']}.
    A aventura escolhida é do tipo '{game_state['chosen_duration_name']}' (aproximadamente {game_state['max_turns']} turnos).

    Informações de Partida Geradas pela IA para esta Aventura (use-as como base fundamental):
    - Objetivo Inicial do Jogador: {game_state['current_objective']}
    - Localização Inicial do Jogador (descreva expandindo isto): {game_state['current_location']}
    - Inventário Inicial do Jogador: {', '.join(game_state['player_inventory'])}

    Instruções para você, GM, ao iniciar a narração da aventura:
    1. Com base RIGOROSA nas "Informações de Partida Geradas pela IA", comece descrevendo vividamente o cenário (2-4 parágrafos).
    2. Integre o 'Objetivo Inicial' na sua descrição de forma natural e imediata.
    3. Apresente uma situação intrigante ou um desafio imediato diretamente relacionado ao objetivo e/ou local.
    4. Termine sua primeira narração convidando o jogador à ação.
    5. Sua primeira resposta deve ser apenas esta narração inicial, sem introduções ou saudações. Não repita o objetivo, localização ou inventário literalmente; incorpore-os na sua prosa.

    GM, comece a aventura para {game_state['player_name']} agora:
    """
    msg_mestre_t = "Mestre (Gemini): Tecendo os fios do destino..."; print(msg_mestre_t); speak_text(msg_mestre_t)
    conversation_history = []
    desc_inicial = mestre_responde(prompt_inicial_contexto)

    if "Erro:" in desc_inicial or "429" in desc_inicial or "quota" in desc_inicial.lower():
        print("\n----------------------------------------------------------------------")
        falha_msg = f"Mestre (Gemini):\n{desc_inicial}"; print(falha_msg); speak_text(desc_inicial)
        print("----------------------------------------------------------------------")
        print("\nVentos da magia desfavoráveis."); game_state["player_name"] = None; return

    print("\n------------------------- SUA AVENTURA COMEÇA! -------------------------")
    print(desc_inicial); speak_text(desc_inicial)
    print("----------------------------------------------------------------------")
    game_state["current_situation"] = desc_inicial
    game_state["world_lore"]["setting_description"] = f"{game_state['current_location']} - {desc_inicial}"
    conversation_history.append({'role': 'user', 'parts': [f"Setup Narrativo: Objetivo='{game_state['current_objective']}', Local='{game_state['current_location']}', Inventário='{', '.join(game_state['player_inventory'])}'."]})
    conversation_history.append({'role': 'model', 'parts': [desc_inicial]})

def rodada_do_jogo():
    global game_state, conversation_history
    game_state["current_turn"] += 1
    print(f"\n--- Turno {game_state['current_turn']}/{game_state['max_turns']} ({game_state['chosen_duration_name']}) ---")

    acao_jogador = input(f"{game_state['player_name']}, o que você faz? (Digite 'sair' para terminar) ")
    if acao_jogador.lower() == 'sair':
        msg = f"\nJornada encerrada, {game_state['player_name']}."; print(msg); speak_text(msg); return False

    game_state["previous_player_actions"] = (game_state["previous_player_actions"] + [acao_jogador])[-3:]

    instrucao_duracao = ""
    ct, mt, cdn = game_state['current_turn'], game_state['max_turns'], game_state['chosen_duration_name']
    if ct == mt: instrucao_duracao = f"ATENÇÃO MESTRE, ÚLTIMO TURNO ({ct}/{mt}) de '{cdn}'! CONCLUA o arco atual. Se possível, deixe um gancho sutil para continuação, MAS SEM perguntar ao jogador sobre isso."
    elif ct == mt - 1 and mt > 1: instrucao_duracao = f"Atenção: Próximo turno ({ct+1}/{mt}) é o último de '{cdn}'. Prepare desfecho."
    elif ct == mt - 2 and mt > 2: instrucao_duracao = f"Atenção: Próximos do fim de '{cdn}' (Turno {ct}/{mt}). Guie para clímax."
    elif ct > mt * 0.60: instrucao_duracao = f"Lembre-se: Aventura '{cdn}' ({mt} turnos), Turno {ct}. Progrida para clímax."

    hist_fmt = ""
    if conversation_history:
        hist_turns = 2
        start_idx = max(0, len(conversation_history) - (hist_turns*2))
        for i in range(start_idx, len(conversation_history)):
            entry = conversation_history[i]
            part_content = str(entry['parts'][0] if isinstance(entry['parts'],list) and entry['parts'] else entry['parts'])
            if entry['role']=='user' and any(s in part_content for s in ["Instruções de Setup", "Setup Narrativo"]): continue
            role_label = "Jogador" if entry['role']=='user' else "Mestre"
            hist_fmt += f"{role_label} fez/respondeu: {part_content}\n"

    prompt_mestre = f"GM de RPG. Jogador: {game_state['player_name']} ({game_state['player_class']}). Aventura: {cdn} ({mt}t), Turno {ct}. {instrucao_duracao}. Histórico Recente:\n{hist_fmt if hist_fmt else 'Nenhum.'}\nSituação Atual: {game_state['current_situation']}. Ação do Jogador: '{acao_jogador}'. Inventário: {', '.join(game_state['player_inventory']) if game_state['player_inventory'] else 'Vazio'}. Objetivo: {game_state['current_objective']}. Instruções: Reaja à ação, avance a narrativa. Considere a duração. Termine com deixa (a menos que seja o final absoluto)."

    conversation_history.append({'role': 'user', 'parts': [acao_jogador]})
    msg_analisando = "\nMestre (Gemini): Analisando..."; print(msg_analisando); speak_text(msg_analisando)
    resp_mestre = mestre_responde(prompt_mestre)

    if "Erro:" in resp_mestre or "429" in resp_mestre or "quota" in resp_mestre.lower():
        print("\n---"); print(f"Mestre (Gemini):\n{resp_mestre}"); speak_text(resp_mestre); print("---\n")
        print("Perturbação mágica. Tente de novo ou 'sair'.")
        if conversation_history and conversation_history[-1]['role'] == 'user': conversation_history.pop()
        game_state["current_turn"] -= 1; return True

    print(f"\nMestre (Gemini):\n{resp_mestre}\n"); speak_text(resp_mestre)
    game_state["current_situation"] = resp_mestre
    conversation_history.append({'role': 'model', 'parts': [resp_mestre]})

    if game_state["current_turn"] >= game_state["max_turns"]:
        fim_planejado_msg = f"Final planejado para '{cdn}' (Turno {ct}/{mt})."; print(fim_planejado_msg); speak_text(fim_planejado_msg)
        can_extend, next_dur_key, curr_dur_key = False, None, None
        for k, v in DURATION_OPTIONS_CONFIG.items():
            if v["id"] == game_state["chosen_duration_id"]: curr_dur_key = k; break
        if curr_dur_key == "1": can_extend, next_dur_key = True, "2"
        elif curr_dur_key == "2": can_extend, next_dur_key = True, "3"

        if can_extend:
            while True:
                next_details = DURATION_OPTIONS_CONFIG[next_dur_key]
                prompt_ext_msg = f"Mestre concluiu o arco. Estender para '{next_details['name']}' (~{next_details['turns']}t totais)? (S/N): "
                choice = input(prompt_ext_msg).upper()
                if choice == "S":
                    game_state.update({"chosen_duration_id": next_details['id'], "chosen_duration_name": next_details['name'], "max_turns": next_details['turns']})
                    prompt_novo_obj = f"Jogador '{game_state['player_name']}' estendeu de '{DURATION_OPTIONS_CONFIG[curr_dur_key]['name']}' para '{game_state['chosen_duration_name']}'. Situação: {game_state['current_situation']}. Sugira novo objetivo/evolução. Responda SÓ com o objetivo."
                    msg_adapt = "Mestre (Gemini): Adaptando o destino..."; print(msg_adapt); speak_text(msg_adapt)
                    novo_obj = mestre_responde(prompt_novo_obj)
                    if "Erro:" not in novo_obj and novo_obj.strip(): game_state["current_objective"] = novo_obj.strip()
                    else:
                        game_state["current_objective"] = f"Explorar desafios de '{game_state['chosen_duration_name']}'."; print(f"Falha ao gerar novo objetivo: {novo_obj}. Usando padrão."); speak_text(f"Falha ao gerar novo objetivo. Objetivo padrão: {game_state['current_objective']}")

                    msg1 = f"\nAventura estendida para: {game_state['chosen_duration_name']}."; print(msg1); speak_text(msg1)
                    msg2 = f"Novo objetivo: {game_state['current_objective']}"; print(msg2); speak_text(msg2)
                    msg3 = f"Até o turno {game_state['max_turns']} para novos mistérios."; print(msg3); speak_text(msg3)
                    return True
                elif choice == "N":
                    msg = "\nEscolheu encerrar. Feitos lembrados!"; print(msg); speak_text(msg); return False
                else:
                    msg = "Inválido. S ou N."; print(msg); speak_text(msg)
        else:
            msg = "\nJornada longa concluída ou sem mais extensões. Obrigado!"; print(msg); speak_text(msg); return False
    return True

# --- Bloco Principal de Execução da Aventura ---
try:
     iniciar_aventura()
     if game_state["player_name"]:
         jogando = True
         while jogando:
             jogando = rodada_do_jogo()
except NameError as e: print(f"Erro de Nome: Função não definida? ({e})")
except SystemExit as e: print(f"Execução interrompida: {e}")
except Exception as e: print(f"Erro inesperado na aventura: {e}")
finally:
     fim_msg = "\n--- Fim da Sessão de Jogo ---"; print(fim_msg)
     # speak_text(fim_msg) # Opcional

nest_asyncio aplicado.
Chave API carregada de arquivo local.
API do Gemini configurada com sucesso.
Modelo Gemini (models/gemini-1.5-flash-latest) configurado!
Bem-vindo, aventureiro! Antes de começarmos nossa jornada, conte-me sobre você.


Qual é o seu nome, ó valente? Jon

Escolha sua vocação entre os heróis de outrora, ou invente a sua:


1. Guerreiro Astuto
2. Maga Perspicaz
3. Ladina Silenciosa
4. Explorador Corajoso
5. Bardo Eloquente
Ou simplesmente digite o nome da classe que deseja inventar.
Digite o número de uma classe da lista ou o nome da sua classe personalizada: Paladino
Você definiu sua classe como: Paladino
Sua classe é: Paladino.



Escolha a duração desejada para esta aventura:


1. Curta (objetivo direto) (~10 turnos)
2. Média (mais exploração) (~20 turnos)
3. Longa (desenvolvimento e clímax) (~40 turnos)
Digite o número (1-3): 1
Duração: Curta (objetivo direto).


...

Em sua bolsa, você carrega: Espada longa +1, escudo de madeira, kit de cura menor



O ar crepita com magia ancestral... Prepare-se, pois sua aventura está prestes a começar!



Mestre (Gemini): Tecendo os fios do destino...



------------------------- SUA AVENTURA COMEÇA! -------------------------
Umidade fria e o cheiro acre de terra úmida e mofo embalam seu nariz enquanto você se encontra na entrada da caverna, a escuridão se abrindo como uma boca faminta nas entranhas da montanha.  A rocha bruta, escorregadia sob seus pés, está coberta por uma fina camada de musgo verde-escuro, que brilha fracamente sob a luz tênue que consegue penetrar a entrada.  Você pode ouvir o eco distante de goteiras, um som constante que acompanha o silêncio opressivo do interior.  A urgência da missão pesa em seus ombros: a filha do prefeito, Elara, foi raptada por goblins e seus rastros levaram até este lugar sombrio, a poucos passos da própria cidadela.  A espada longa +1, firme em sua mão, parece a única luz em meio a esta escuridão ameaçadora.

A entrada se estreita bruscamente após alguns passos, formando um túnel baixo e apertado.  Você percebe marcas frescas de arrasto na terra úmida, indicando o caminho que os goblins t

----------------------------------------------------------------------

--- Turno 1/10 (Curta (objetivo direto)) ---
